In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

In [ ]:
data = np.load('visualize_data.npz')
print(data.keys())
weight = data[data.keys()[0]]
print(weight.shape)

In [ ]:
def img_norm(data, mark):
    line_coordinate = np.argwhere(data == mark)
       
    if np.min(data) > 0 :
        data -= np.min(data)
    else :
        data += np.abs(np.min(data))
    
    data / np.max(data)    
    
    for coor in line_coordinate :
        data[coor[0], coor[1]] = 0
    
    return data

In [ ]:
#convolution filter visualization
keys = ['c1', 'c2']
fold = weight.shape[0]
epoch = weight.shape[1]
mark = 24

fig = plt.figure(frameon=False)

for key_idx in keys:
    #Reshape filter data
    fold_tmp = []
    for fold_idx in range(fold) :
        epoch_tmp = []
        for epoch_idx in range(epoch) :
            epoch_tmp.append(weight[fold_idx, epoch_idx][key_idx])
        fold_tmp.append(epoch_tmp)
    fold_tmp = np.array(fold_tmp)
    print(fold_tmp.shape)
    k_size = fold_tmp.shape[2]
    
    draw_data = fold_tmp
    
    #Draw figure
    for fold_idx in range(fold) :
        for epoch_idx in range(epoch) :
            for out_idx in range(draw_data.shape[5]):
                for in_idx in range(draw_data.shape[4]):
                    if in_idx == 0 :
                        in_tmp = draw_data[fold_idx, epoch_idx, :, :, in_idx, out_idx].reshape(k_size*k_size,1)
                    else :
                        in_tmp = np.hstack((in_tmp, draw_data[fold_idx, epoch_idx, :, :, in_idx, out_idx].reshape(k_size*k_size,1)))
                if out_idx == 0 :
                    out_tmp = in_tmp
                else :
                    out_tmp = np.vstack((out_tmp, np.zeros((1,out_tmp.shape[1]))+mark))
                    out_tmp = np.vstack((out_tmp, in_tmp))
            if epoch_idx == 0 :
                epoch_tmp = out_tmp
            else :
                epoch_tmp = np.hstack((epoch_tmp, np.zeros((epoch_tmp.shape[0], 1))+mark))
                epoch_tmp = np.hstack((epoch_tmp, out_tmp))
        print(epoch_tmp.shape)
        conv_img = img_norm(epoch_tmp, mark)
                
        fname = str(key_idx) + "_fold"+ str(fold_idx+1) + ".png"
        plt.imsave(fname,conv_img, cmap='gray')
        print("save file:",fname)